<a href="https://colab.research.google.com/github/mijanr/TimeSeries/blob/master/Ford_data_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [62]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

#tensorboard
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter('runs/ford_data_classification')

In [63]:
root_url = "https://raw.githubusercontent.com/hfawaz/cd-diagram/master/FordA/"

In [64]:
def readucr(filename):
    data = np.loadtxt(filename, delimiter="\t")
    y = data[:, 0]
    x = data[:, 1:]
    return x, y.astype(int)

In [65]:
x_train, y_train = readucr(root_url + "FordA_TRAIN.tsv")
x_test, y_test = readucr(root_url + "FordA_TEST.tsv")

In [66]:
#convert y to 0 and 1
y_train[y_train == -1] = 0
y_test[y_test == -1] = 0

In [67]:
#normalize x
x_train = MinMaxScaler().fit_transform(x_train)
x_test = MinMaxScaler().fit_transform(x_test)

In [68]:
#expand dimension
x_train = np.expand_dims(x_train, axis=1)
x_test = np.expand_dims(x_test, axis=1)

In [69]:
#check shapes
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(3601, 1, 500) (3601,) (1320, 1, 500) (1320,)


In [70]:
#convert to torch tensors
x_train = torch.from_numpy(x_train).float()
y_train = torch.from_numpy(y_train).long()
x_test = torch.from_numpy(x_test).float()
y_test = torch.from_numpy(y_test).long()

In [71]:
#convert to torch dataset
train = TensorDataset(x_train, y_train)
test = TensorDataset(x_test, y_test)
#convert to dataloader
train_loader = DataLoader(train, batch_size=64, shuffle=True)
test_loader = DataLoader(test, batch_size=64, shuffle=False)

In [72]:
#check shapes
for x, y in train_loader:
    print(x.shape, y.shape)
    break

torch.Size([64, 1, 500]) torch.Size([64])


In [73]:
#1D CNN
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.sequential = nn.Sequential(
            nn.Conv1d(1, 64, 3),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Conv1d(64, 128, 3),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Conv1d(128, 128, 3),
            nn.ReLU(),
            nn.MaxPool1d(2),
            #flatten
            nn.Flatten(),
            nn.LazyLinear(out_features=128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 2)
        )
    def forward(self, x):
        return self.sequential(x)

In [74]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = CNN().to(device)

cuda


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [75]:
#feed a sample to the model
for x, y in train_loader:
    x = x.to(device)
    print(model(x).shape)
    break

torch.Size([64, 2])


In [76]:
#loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [77]:
#training
n_epochs = 200
for epoch in range(n_epochs):
    train_loss = 0.0
    for x, y in train_loader:
        x = x.to(device)
        y = y.to(device)
        optimizer.zero_grad()
        outputs = model(x)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * x.size(0)
    train_loss = train_loss / len(train_loader.dataset)
    if epoch % 100 == 0:
        print("Epoch: {} \tTraining Loss: {:.6f}".format(epoch, train_loss))
    #log loss
    writer.add_scalar("training loss", train_loss, epoch)
    #log accuracy
    correct = 0
    total = 0
    with torch.no_grad():
        for x, y in test_loader:
            x = x.to(device)
            y = y.to(device)
            outputs = model(x)
            _, predicted = torch.max(outputs.data, 1)
            total += y.size(0)
            correct += (predicted == y).sum().item()
    accuracy = 100 * correct / total
    writer.add_scalar("accuracy", accuracy, epoch)
    #close writer
    writer.close()

Epoch: 0 	Training Loss: 0.693331
Epoch: 100 	Training Loss: 0.000003


In [78]:
#check the mdodel on test data
with torch.no_grad():
    correct = 0
    total = 0
    for x, y in test_loader:
        x = x.to(device)
        y = y.to(device)
        outputs = model(x)
        _, predicted = torch.max(outputs.data, 1)
        total += y.size(0)
        correct += (predicted == y).sum().item()
    print(f'Accuracy: {100 * correct / total} %')

Accuracy: 86.5909090909091 %
